<a href="https://colab.research.google.com/github/PaulToronto/Stanford-Andrew-Ng-Machine-Learning-Specialization/blob/main/Gradient_Descent_with_Sympy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent with `sympy`

## Imports

In [ ]:
import sympy as sym
import pandas as pd
import numpy as np
from math import ceil

## Symbols

In [ ]:
# training set data
x11, x12, x13, x14 = sym.symbols('x_{11} x_{12} x_{13} x_{14}')
x21, x22, x23, x24 = sym.symbols('x_{21} x_{22} x_{23} x_{24}')
x31, x32, x33, x34 = sym.symbols('x_{31} x_{32} x_{33} x_{34}')

# target
y1, y2, y3 = sym.symbols('y_1 y_2 y_3')

# weights and bias
w1, w2, w3, w4, b = sym.symbols('w_1 w_2 w_3 w_4 b')

# weight for simple linear regression
m1 = sym.symbols('m_1')

## Toy Datasets

### Dataset with symbols

In [ ]:
data = pd.DataFrame({'feature1': [x11, x21, x31],
                     'feature2': [x12, x22, x32],
                     'feature3': [x13, x23, x33],
                     'feature4': [x14, x24, x34],
                     'target': [y1, y2, y3]})

data

,feature1,feature2,feature3,feature4,target
0,x_{11},x_{12},x_{13},x_{14},y_1
1,x_{21},x_{22},x_{23},x_{24},y_2
2,x_{31},x_{32},x_{33},x_{34},y_3


### Dataset with numbers

In [ ]:
data_num = pd.DataFrame({'feature1': [2104, 1416, 852],
                         'feature2': [5, 3, 2],
                         'feature3': [1, 2, 1],
                         'feature4': [45, 40, 35],
                         'target': [460, 232, 178]})

data_num

,feature1,feature2,feature3,feature4,target
0,2104,5,1,45,460
1,1416,3,2,40,232
2,852,2,1,35,178


In [ ]:
# optimal w and b for testing
wn_best = sym.Matrix(np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618]))
bn_best = 785.1811367994083
wn_best

Matrix([
[  0.39133535],
[ 18.75376741],
[-53.36032453],
[-26.42131618]])

### Dataset for simple linear regression

In [ ]:
data_simple = pd.DataFrame({'feature': [1, 2],
                            'target': [300, 500]})
data_simple

,feature,target
0,1,300
1,2,500


In [ ]:
# optimal m and b for testing
ms_best = 200
bs_best = 100

## Using `sympy` Matrices to train the model

`X_train` is represented by the matrix $\mathbf{X}$

$$
\mathbf{X} = \begin{bmatrix}
x_{11} & x_{12} & x_{13} & x_{14}\\
x_{21} & x_{22} & x_{23} & x_{24}\\
x_{31} & x_{32} & x_{33} & x_{34}\\
\end{bmatrix}
$$

`y_train` is represented by the column matrix $\mathbf{y}$.

$$
\mathbf{y} = \begin{bmatrix}y_1\\y_2\\y_3\end{bmatrix}
$$

The weights of the model are represented by the matrix $\mathbf{w}$ the bias is represented by the scalar $b$.

$$
\mathbf{w} = \begin{bmatrix}w_1\\w_2\\w_3\\w_4\end{bmatrix}
$$

The goal of linear regression is to find the values for $\mathbf{w}$ and $b$ that minimize the cost.

$$
\begin{align}
\mathbf{X}\mathbf{w} + b &= \mathbf{y} \\
\begin{bmatrix}
x_{11} & x_{12} & x_{13} & x_{14}\\
x_{21} & x_{22} & x_{23} & x_{24}\\
x_{31} & x_{32} & x_{33} & x_{34}\\
\end{bmatrix}
\begin{bmatrix}w_1\\w_2\\w_3\\w_4\end{bmatrix} +
b\begin{bmatrix}1\\1\\1\end{bmatrix} &= \begin{bmatrix}y_1\\y_2\\y_3\end{bmatrix} \\
\begin{bmatrix}
w_1x_{11} + w_2x_{12} + w_3x_{13} + w_4x_{14} + b\\
w_1x_{21} + w_2x_{22} + w_3x_{23} + w_4x_{24} + b\\
w_1x_{31} + w_2x_{32} + w_3x_{33} + w_4x_{34} + b
\end{bmatrix} &= \begin{bmatrix}y_1\\y_2\\y_3\end{bmatrix}
\end{align}
$$



In [ ]:
X = sym.Matrix(data.drop('target', axis=1))
y = sym.Matrix(data['target'])
w = sym.Matrix([w1, w2, w3, w4])

X @ w + b * sym.Matrix([1, 1, 1])

Matrix([
[b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14}],
[b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24}],
[b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34}]])

In [ ]:
y

Matrix([
[y_1],
[y_2],
[y_3]])

In [ ]:
# dataset with numbers
Xn = sym.Matrix(data_num.drop('target', axis=1))
yn = sym.Matrix(data_num['target'])

In [ ]:
# dataset for simple linear regression
Xs = sym.Matrix(data_simple.drop('target', axis=1))
ys = sym.Matrix(data_simple['target'])

## The Model Prediction

$$
f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w}\cdot\mathbf{x}^{(i)} + b
$$

where

- $\mathbf{x}$ is a vector representing the $i^{th}$ row of $\mathbf{X}$
- $\mathbf{w}$ is vector containing the weights of the model
- $b$ is a scalar representing the bias

In [ ]:
# `X * w` is used instead of `X @ w`
#   so that the function also works
#   for simple linear regression
def f_wb(X, w, b):
    m = X.shape[0]
    pred = X * w + b * sym.ones(m, 1)
    return pred

In [ ]:
# works with a single row
f_wb(X[0,:], w, b)

Matrix([[b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14}]])

In [ ]:
f_wb(Xn[0,:], w, b)

Matrix([[b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4]])

In [ ]:
# works with multiple rows
f_wb(X, w, b)

Matrix([
[b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14}],
[b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24}],
[b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34}]])

In [ ]:
f_wb(Xn, w, b)

Matrix([
[  b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4],
[b + 1416*w_1 + 3*w_2 + 2*w_3 + 40*w_4],
[   b + 852*w_1 + 2*w_2 + w_3 + 35*w_4]])

In [ ]:
# works with simple linear regression
f_wb(X[:,0], m1, b)

Matrix([
[b + m_1*x_{11}],
[b + m_1*x_{21}],
[b + m_1*x_{31}]])

In [ ]:
f_wb(Xs, m1, b)

Matrix([
[  b + m_1],
[b + 2*m_1]])

## The Cost Function

$$
J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2
$$

In [ ]:
def compute_cost_loop(X, y, w, b):
    m = X.shape[0]
    cost = sym.Matrix([0.0])

    for i in range(m):
        f_wb_i = f_wb(X[i,:], w, b)
        cost = cost + (f_wb_i - sym.Matrix([y[i]])).applyfunc(lambda x: x**2)
    cost = cost / (2 * m)
    return cost[0]

In [ ]:
# works for multiple linear regression
compute_cost_loop(X, y, w, b)

(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)**2/6 + (b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)**2/6 + (b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)**2/6

In [ ]:
compute_cost_loop(Xn, yn, w, b)

(b + 852*w_1 + 2*w_2 + w_3 + 35*w_4 - 178)**2/6 + (b + 1416*w_1 + 3*w_2 + 2*w_3 + 40*w_4 - 232)**2/6 + (b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4 - 460)**2/6

In [ ]:
# works for simple linear regression
compute_cost_loop(X[:,0], y, m1, b)

(b + m_1*x_{11} - y_1)**2/6 + (b + m_1*x_{21} - y_2)**2/6 + (b + m_1*x_{31} - y_3)**2/6

In [ ]:
compute_cost_loop(Xs, ys, m1, b)

(b + m_1 - 300)**2/4 + (b + 2*m_1 - 500)**2/4

This can also be implemented without the loop.

In [ ]:
def compute_cost(X, y, w, b):
    m = X.shape[0]
    pred = f_wb(X, w, b)
    cost = sum((pred - y).applyfunc(lambda x: x**2)) / (2 * m)
    return cost

In [ ]:
# works for mulitiple linear regression
compute_cost(X, y, w, b)

(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)**2/6 + (b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)**2/6 + (b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)**2/6

In [ ]:
compute_cost(Xn, yn, w, b)

(b + 852*w_1 + 2*w_2 + w_3 + 35*w_4 - 178)**2/6 + (b + 1416*w_1 + 3*w_2 + 2*w_3 + 40*w_4 - 232)**2/6 + (b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4 - 460)**2/6

In [ ]:
# test with optimal parameters
compute_cost(Xn, yn, wn_best, bn_best)

1.55789044289666e-12

In [ ]:
# works for simple linear regression
compute_cost(X[:,0], y, m1, b)

(b + m_1*x_{11} - y_1)**2/6 + (b + m_1*x_{21} - y_2)**2/6 + (b + m_1*x_{31} - y_3)**2/6

In [ ]:
compute_cost(Xs, ys, m1, b)

(b + m_1 - 300)**2/4 + (b + 2*m_1 - 500)**2/4

In [ ]:
# test with optimal parameters
compute_cost(Xs, ys, ms_best, bs_best)

0

### Speed Comparison of `compute_cost_loop` and `compute_cost`

#### `compute_cost_loop`

In [ ]:
%%timeit -r7 -n1000
compute_cost_loop(X, y, w, b)

2.41 ms ± 995 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_cost_loop(Xn, yn, w, b)

1.51 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_cost_loop(Xn, yn, wn_best, bn_best)

1.93 ms ± 316 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_cost_loop(Xs, ys, ms_best, bs_best)

694 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### `compute_cost`

In [ ]:
%%timeit -r7 -n1000
compute_cost(X, y, w, b)

464 µs ± 109 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_cost(Xn, yn, w, b)

505 µs ± 139 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_cost(Xn, yn, wn_best, bn_best)

732 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_cost(Xs, ys, ms_best, bs_best)

214 µs ± 8.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## The Gradient

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})
\end{align}
$$

In [ ]:
def compute_gradient_loop(X, y, w, b):
    m, n = X.shape

    dj_dw = sym.zeros(n, 1)
    dj_db = 0.0

    for i in range(m):
        err = f_wb(X[i,:], w, b)[0] - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + (err * X[i, j])
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw

In [ ]:
# works for mulitiple linear regression
dj_db, dj_dw = compute_gradient_loop(X, y, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + w_1*x_{11} + w_1*x_{21} + w_1*x_{31} + w_2*x_{12} + w_2*x_{22} + w_2*x_{32} + w_3*x_{13} + w_3*x_{23} + w_3*x_{33} + w_4*x_{14} + w_4*x_{24} + w_4*x_{34} - y_1 - y_2 - y_3)/3

Matrix([
[x_{11}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{21}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{31}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{12}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{22}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{32}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{13}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{23}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{33}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{14}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{24}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{34}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3]])

In [ ]:
dj_db, dj_dw = compute_gradient_loop(Xn, yn, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + 4372*w_1 + 10*w_2 + 4*w_3 + 120*w_4 - 870)/3

Matrix([
[4372*b/3 + 7157776*w_1/3 + 16472*w_2/3 + 5788*w_3/3 + 60380*w_4 - 1448008/3],
[            10*b/3 + 16472*w_1/3 + 38*w_2/3 + 13*w_3/3 + 415*w_4/3 - 3352/3],
[                 4*b/3 + 5788*w_1/3 + 13*w_2/3 + 2*w_3 + 160*w_4/3 - 1102/3],
[              40*b + 60380*w_1 + 415*w_2/3 + 160*w_3/3 + 4850*w_4/3 - 12070]])

In [ ]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient_loop(Xn, yn, wn_best, bn_best)
display(dj_db)
display(dj_dw)

-1.67392515019552e-6

Matrix([
[-0.00272623577196403],
[-6.27197262777675e-6],
[-2.21745578225333e-6],
[-6.92403390682254e-5]])

In [ ]:
# works for simple linear regression
dj_db, dj_dw = compute_gradient_loop(X[:,0], y, m1, b)
display(dj_db.factor())
display(dj_dw)

(3*b + m_1*x_{11} + m_1*x_{21} + m_1*x_{31} - y_1 - y_2 - y_3)/3

Matrix([[x_{11}*(b + m_1*x_{11} - y_1)/3 + x_{21}*(b + m_1*x_{21} - y_2)/3 + x_{31}*(b + m_1*x_{31} - y_3)/3]])

In [ ]:
dj_db, dj_dw = compute_gradient_loop(Xs, ys, m1, b)
display(dj_db.factor())
display(dj_dw)

(2*b + 3*m_1 - 800)/2

Matrix([[3*b/2 + 5*m_1/2 - 650]])

In [ ]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient_loop(Xs, ys, ms_best, bs_best)
display(dj_db)
display(dj_dw)

0

Matrix([[0]])

This can also be implemented without the loop.

In [ ]:
def compute_gradient(X, y, w, b):
    m, n = X.shape

    y_pred = f_wb(X, w, b)
    err = y_pred - y

    dj_dw = (X.T @ err) / m
    dj_db = sum(err) / m

    return dj_db, dj_dw

In [ ]:
# works for mulitiple linear regression
dj_db, dj_dw = compute_gradient(X, y, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + w_1*x_{11} + w_1*x_{21} + w_1*x_{31} + w_2*x_{12} + w_2*x_{22} + w_2*x_{32} + w_3*x_{13} + w_3*x_{23} + w_3*x_{33} + w_4*x_{14} + w_4*x_{24} + w_4*x_{34} - y_1 - y_2 - y_3)/3

Matrix([
[x_{11}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{21}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{31}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{12}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{22}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{32}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{13}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{23}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{33}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{14}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{24}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{34}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3]])

In [ ]:
dj_db, dj_dw = compute_gradient(Xn, yn, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + 4372*w_1 + 10*w_2 + 4*w_3 + 120*w_4 - 870)/3

Matrix([
[4372*b/3 + 7157776*w_1/3 + 16472*w_2/3 + 5788*w_3/3 + 60380*w_4 - 1448008/3],
[            10*b/3 + 16472*w_1/3 + 38*w_2/3 + 13*w_3/3 + 415*w_4/3 - 3352/3],
[                 4*b/3 + 5788*w_1/3 + 13*w_2/3 + 2*w_3 + 160*w_4/3 - 1102/3],
[              40*b + 60380*w_1 + 415*w_2/3 + 160*w_3/3 + 4850*w_4/3 - 12070]])

In [ ]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient(Xn, yn, wn_best, bn_best)
display(dj_db)
display(dj_dw)

-1.67392515019552e-6

Matrix([
[-0.00272623577196403],
[-6.27197262777675e-6],
[-2.21745578225333e-6],
[-6.92403390682254e-5]])

In [ ]:
# works for simple linear regression
dj_db, dj_dw = compute_gradient(X[:,0], y, m1, b)
display(dj_db.factor())
display(dj_dw)

(3*b + m_1*x_{11} + m_1*x_{21} + m_1*x_{31} - y_1 - y_2 - y_3)/3

Matrix([[x_{11}*(b + m_1*x_{11} - y_1)/3 + x_{21}*(b + m_1*x_{21} - y_2)/3 + x_{31}*(b + m_1*x_{31} - y_3)/3]])

In [ ]:
dj_db, dj_dw = compute_gradient(Xs, ys, m1, b)
display(dj_db.factor())
display(dj_dw)

(2*b + 3*m_1 - 800)/2

Matrix([[3*b/2 + 5*m_1/2 - 650]])

In [ ]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient(Xs, ys, ms_best, bs_best)
display(dj_db)
display(dj_dw)

0

Matrix([[0]])

### Speed Comparison of `compute_gradient_loop` and `compute_gradient`

#### `compute_gradient_loop`

In [ ]:
%%timeit -r7 -n1000
compute_gradient_loop(X, y, w, b)

1.12 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient_loop(Xn, yn, w, b)

1.42 ms ± 290 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient_loop(Xn, yn, wn_best, bn_best)

1.78 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient_loop(Xs, ys, ms_best, bs_best)

401 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### `compute_gradient`

In [ ]:
%%timeit -r7 -n1000
compute_gradient(X, y, w, b)

647 µs ± 128 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient(Xn, yn, w, b)

747 µs ± 205 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient(Xn, yn, wn_best, bn_best)

981 µs ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient(Xs, ys, ms_best, bs_best)

299 µs ± 62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Using `sympy` to compute the gradient

This only works when the parameters, $\mathbf{w}$ and $b$ are variables.

In [ ]:
def compute_gradient_sympy(X, y, w, b):
    cost = compute_cost(X, y, w, b)
    w = sym.Matrix([w]) # so it works with simple regression

    dj_dw = w.applyfunc(lambda x: sym.diff(cost, x))
    dj_db = sym.diff(cost, b)

    return dj_db, dj_dw

In [ ]:
# works for mulitiple linear regression
dj_db, dj_dw = compute_gradient_sympy(X, y, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + w_1*x_{11} + w_1*x_{21} + w_1*x_{31} + w_2*x_{12} + w_2*x_{22} + w_2*x_{32} + w_3*x_{13} + w_3*x_{23} + w_3*x_{33} + w_4*x_{14} + w_4*x_{24} + w_4*x_{34} - y_1 - y_2 - y_3)/3

Matrix([
[x_{11}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{21}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{31}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{12}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{22}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{32}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{13}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{23}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{33}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{14}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{24}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{34}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3]])

In [ ]:
dj_db, dj_dw = compute_gradient_sympy(Xn, yn, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + 4372*w_1 + 10*w_2 + 4*w_3 + 120*w_4 - 870)/3

Matrix([
[4372*b/3 + 7157776*w_1/3 + 16472*w_2/3 + 5788*w_3/3 + 60380*w_4 - 1448008/3],
[            10*b/3 + 16472*w_1/3 + 38*w_2/3 + 13*w_3/3 + 415*w_4/3 - 3352/3],
[                 4*b/3 + 5788*w_1/3 + 13*w_2/3 + 2*w_3 + 160*w_4/3 - 1102/3],
[              40*b + 60380*w_1 + 415*w_2/3 + 160*w_3/3 + 4850*w_4/3 - 12070]])

In [ ]:
# works for simple linear regression
dj_db, dj_dw = compute_gradient_sympy(X[:,0], y, m1, b)
display(dj_db.factor())
display(dj_dw)

(3*b + m_1*x_{11} + m_1*x_{21} + m_1*x_{31} - y_1 - y_2 - y_3)/3

Matrix([[x_{11}*(b + m_1*x_{11} - y_1)/3 + x_{21}*(b + m_1*x_{21} - y_2)/3 + x_{31}*(b + m_1*x_{31} - y_3)/3]])

In [ ]:
dj_db, dj_dw = compute_gradient_sympy(Xs, ys, m1, b)
display(dj_db.factor())
display(dj_dw)

(2*b + 3*m_1 - 800)/2

Matrix([[3*b/2 + 5*m_1/2 - 650]])

#### Speed Comparison of `compute_gradient` and `compute_gradient_sympy`

`compute_gradient_sympy` is much slower as expected.

##### `compute_gradient`

In [ ]:
%%timeit -r7 -n1000
compute_gradient(X, y, w, b)

502 µs ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient(Xn, yn, w, b)

583 µs ± 31.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient(X[:,0], y, m1, b)

395 µs ± 79.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient(Xs, ys, m1, b)

451 µs ± 109 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### `compute_gradient_sympy`

In [ ]:
%%timeit -r7 -n1000
compute_gradient_sympy(X, y, w, b)

1.92 ms ± 412 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient_sympy(Xn, yn, w, b)

1.68 ms ± 248 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient_sympy(X[:,0], y, m1, b)

793 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit -r7 -n1000
compute_gradient_sympy(Xs, ys, m1, b)

732 µs ± 140 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Gradient Descent

In [ ]:
def gradient_descent(X, y, w, b, f_cost, f_gradient, alpha, num_iters):

    J_history = []

    for i in range(num_iters):
        dj_db, dj_dw = f_gradient(X, y, w, b)

        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        J_history.append(f_cost(X, y, w, b))

        # print cost
        if i % ceil(num_iters / 10) == 0:
            print(f'Iteration {i:4d}: Cost {J_history[-1]:8.2f}')

    return w, b, J_history